## Data prep and modeling

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC, binary_accuracy
from tensorflow.keras.callbacks import LearningRateScheduler
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, split=['train'], shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incomplete1D5QZB/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [3]:
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2
DATASET_SIZE = len(dataset[0])

def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IM_SIZE = 224

def resize_rescale(image, label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)
# test_dataset = test_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)

In [4]:
dropout_rate = 0.2
regularization_rate = 0.01

model = tf.keras.Sequential([InputLayer(input_shape=(IM_SIZE, IM_SIZE, 3)),

                             Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_regularizer=L2(regularization_rate)),
                             BatchNormalization(),
                             MaxPool2D(pool_size=2, strides=2),
                             Dropout(rate=dropout_rate),

                             Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_regularizer=L2(regularization_rate)),
                             BatchNormalization(),
                             MaxPool2D(pool_size=2, strides=2),

                             Flatten(),

                             Dense(100, activation='sigmoid', kernel_regularizer=L2(regularization_rate)), #
                             BatchNormalization(),
                             Dropout(rate=dropout_rate),

                             Dense(10, activation='sigmoid', kernel_regularizer=L2(regularization_rate)),
                             BatchNormalization(),
                             Dense(1, activation='sigmoid')]) # 0 or 1
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 6)       168       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 6)      24        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 6)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 111, 111, 6)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_1 (Batc  (None, 109, 109, 16)     6

## Custom training loop

Gradient descent in training loop

![](https://miro.medium.com/v2/1*TKwMAQnyj_CJgZeT1dgg2g.png)

In [6]:
def custom_bce(y_true, y_pred):
  bce = BinaryCrossentropy()
  return bce(y_true, y_pred)

In [7]:
OPTIMIZER = Adam(learning_rate=0.001)
METRIC = BinaryAccuracy()
METRIC_VAL = BinaryAccuracy()
EPOCHS = 2

In [14]:
# Changing to graph mode where the computation is expensive => Faster!
@tf.function
def training_block(x_batch, y_batch):
  # Need gradients to update
    with tf.GradientTape() as tape:
      y_pred = model(x_batch, training=True)
      loss = custom_bce(y_batch, y_pred)

    partial_derivatives = tape.gradient(loss, model.trainable_weights)
    OPTIMIZER.apply_gradients(zip(partial_derivatives, model.trainable_weights))

    METRIC.update_state(y_batch, y_pred)
    return loss

@tf.function
def validation_block(x_batch, y_batch):
  # Validation after each epoch
  y_pred = model(x_batch, training=False)
  loss_val = custom_bce(y_batch, y_pred)
  METRIC_VAL.update_state(y_batch, y_pred)
  return loss_val

In [16]:
def training_loop(model, loss_function, METRIC, METRIC_VAL, optimizer, train_dataset, val_dataset, EPOCHS):
  for epoch in range(EPOCHS):
    print(f"Training starts for epoch {epoch}")
    # Get every batch in the dataset
    for step, (x_batch, y_batch) in enumerate(train_dataset):

      loss = training_block(x_batch, y_batch)

    print("Training loss:", loss)
    print("Training accuracy:", METRIC.result())
    METRIC.reset_state()

    for x_batch_val, y_batch_val in val_dataset:
      loss_val = validation_block(x_batch_val, y_batch_val)

    print("Validation Loss:", loss_val)
    print("Validation accuracy:", METRIC_VAL.result())
    METRIC_VAL.reset_state()

In [17]:
training_loop(model, custom_bce, METRIC, METRIC_VAL, OPTIMIZER, train_dataset, val_dataset, EPOCHS)

Training starts for epoch 0
Training loss: tf.Tensor(0.6863234, shape=(), dtype=float32)
Training accuracy: tf.Tensor(0.54245794, shape=(), dtype=float32)
Validation Loss: tf.Tensor(0.695051, shape=(), dtype=float32)
Validation accuracy: tf.Tensor(0.53982943, shape=(), dtype=float32)
Training starts for epoch 1
Training loss: tf.Tensor(0.71721894, shape=(), dtype=float32)
Training accuracy: tf.Tensor(0.5834644, shape=(), dtype=float32)
Validation Loss: tf.Tensor(0.8159119, shape=(), dtype=float32)
Validation accuracy: tf.Tensor(0.59317726, shape=(), dtype=float32)
